Import des différentes librairies et configuration des widgets

In [1]:
import tweepy
import squarify 
from geopy.geocoders import Nominatim
import folium
import json_lines
import json
import time
import datetime
import random
import itertools
import os
import matplotlib.pyplot as plt
from texttable import Texttable
import jsonpickle
from ipywidgets import widgets
from IPython.display import display
import pandas as pd, requests
import certifi
import subprocess
from IPython.display import clear_output
# pip install tweepy json_lines squarify folium geopy panda

def main(b):
    #try:
        w_progress_bar.value = 1
        w_progress_legend.value = "Performing API query"
        global TIME, OUTPUT_TWEET_FILENAME, TWEET_JSON_FILENAME
        TIME=time.time()
        OUTPUT_TWEET_FILENAME = "output_tweet_{}.json".format(TIME)
        TWEET_JSON_FILENAME = "input_tweet_{}.json".format(TIME)
        download_tweets()
        run_bash()
        w_progress_bar.value += 1
        w_progress_legend.value = "Processing data for map ..."
        format_to_map()
        w_progress_bar.value += 1
        w_progress_legend.value = "Done !"
    #except Exception as e:
       # w_progress_legend.value = "There was a mistake. Interruption"
       # print(str(e))
# Define all widgets
w_type = widgets.ToggleButtons(
    options=["recent", "popular", "mixed"],
    value="recent",
    description="Tweet type",
    tooltips=['Only the most recent tweets', 'Select the most popular tweets', 'Both recent and popular tweets'],
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    disabled=False,
)

w_keyword = widgets.Text(
    value='#Trump',
    placeholder='keyword or hashtag',
    description='Keyword',
    disabled=False   
)

w_language = widgets.ToggleButtons(
    options=["en", "fr", "de"],
    value="en",
    description="Language",
    tooltips=["English", "French", "German"],
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    disabled=False,
)

w_rts = widgets.Checkbox(
    value=False,
    description='Exclude retweets ?',
    disabled=False
)

w_count = widgets.IntSlider(
    value=100,
    max=1000,
    description='Nb of tweets',
)

w_coordinates = widgets.Text(
    placeholder='Coordinates',
    description='Coordinates',
    disabled=False   
)

w_radius = widgets.IntText(
    value=100,
    placeholder='Number if kilometers',
    description= 'Radius (km)',
    disabled=False   
)

today = datetime.datetime.now()
dates = [today - datetime.timedelta(days=10-i) for i in range(10)]
options = [(date.strftime('%d/%m')) for date in dates]
w_dates = widgets.SelectionRangeSlider(
    options=options,
    index=(0,9),
    description='Date range',
    disabled=False
)

button_query = widgets.Button(description="Perform the query and the analysis")
button_query.on_click(main)
    
w_progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=7,
    step=1,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)

w_progress_legend = widgets.HTML(
    value="Waiting for input ... ",
    placeholder='Some HTML',
    description='Action : ',
)


Configuration des noms de fichiers utilisés par défaut

In [2]:
OUTPUT_TWEET_FILENAME="output_test.json"
TWEET_JSON_FILENAME="input_test.json"

Collecte des données twitter et stockage dans un json 'tweets_time.json'

In [3]:
def download_tweets():
    # Set API_key
    API_key = json.load(open("API_key.txt", "r"))
    auth = tweepy.OAuthHandler(API_key["consumer_key"], API_key["consumer_secret"])
    auth.set_access_token(API_key["access_token"], API_key["access_token_secret"])
    api = tweepy.API(auth)

    # If results from a specific ID onwards are reqd, set since_id to that ID.
    # else default to no lower limit, go as far back as API allows
    sinceId = None

    # If results only below a specific ID are, set max_id to that ID.
    # else default to no upper limit, start from the most recent tweet matching the search query.
    max_id = -1

    # Get values from widgets
    maxTweets = w_count.value
    searchQuery = w_keyword.value
    language = w_language.value
    year = str(datetime.datetime.now().year)
    rts = " -filter:retweets" if w_rts.value else ""

    since_date = datetime.datetime.strptime(w_dates.value[0], '%d/%m')
    until_date = datetime.datetime.strptime(w_dates.value[1], '%d/%m')
    since = year + "-" + since_date.strftime('%m-%d')
    until = year + "-" + until_date.strftime('%m-%d')

    tweetsPerQry = 100
    result_type = w_type.value
    tweet_mode = "extended"

    args= {
        "q" : searchQuery + rts + " -filter:replies",
        "until" : until,
        "since" : since,
        "lang" : language,
        "result_type" : result_type,
        "tweet_mode" : tweet_mode,
    }
    if sinceId:
        args["since_id"] = sinceId
    if w_coordinates.value and w_radius.value:
        args["geocode"] = str(w_coordinates.value) + " " + str(w_radius.value) + "km"

    tweetCount = 0
    #print("Downloading max {0} tweets".format(maxTweets))

    filename = "tweets-database/" + TWEET_JSON_FILENAME
    with open(filename, 'w') as f:
        while tweetCount < maxTweets:
            args["count"] =  min(tweetsPerQry, maxTweets-tweetCount)
            if (max_id > 0):
                args["max_id"] = str(max_id - 1)
            try:
                new_tweets = api.search(**args)
                if not new_tweets:
                    #print("No more tweets found")
                    break
                for tweet in new_tweets:
                    f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
                tweetCount += len(new_tweets)
                #print("Downloaded {0} tweets".format(tweetCount))
                max_id = new_tweets[-1].id
            except tweepy.TweepError as e:
                # Just exit if any error
                print("some error : " + str(e))
                break

    #print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, filename))

Envoie des tweets collectés sur le datacenter et lancement de l'analyse

In [4]:
def run_bash():
    commands = [
'''cp tweets-database/{input} Algo1/{input}
cd Algo1
sbt "put {input}"''',
        
'''cd Algo1
rm -rf {input}
sbt "submit {input}"''',
        
'''cd Algo1
sbt "getOutput 'defaultoutput.json'"''',
        
'''cd Algo1
cp defaultoutput.json ../tweets-database/{output}
rm -rf defaultoutput.json'''
    ]
    
    actions = ["Uploading tweets file...", 
               "Submitting file...", 
               "Running algorithm...",
               "Saving output..."]
    commands = [string.format(input=TWEET_JSON_FILENAME, output=OUTPUT_TWEET_FILENAME) for string in commands]
    for i, cmd in enumerate(commands):
        w_progress_bar.value += 1
        w_progress_legend.value = actions[i]
        subprocess.call(cmd, shell=True) 
    
    #""" Do not remove this line, this corrects a bug in Jupyter core

Formatting data for map

In [5]:
def format_to_map():
    global map_main, map_color
    file_json = open("tweets-database/" + OUTPUT_TWEET_FILENAME,"r", encoding="utf8");
    reader = json_lines.reader(file_json)
    reader = list(reader)
    d = {"af":"AFG",	"al":"ALB",	"dz":"DZA",	"as":"ASM",	"ad":"AND",	"ao":"AGO",	"ai":"AIA",	"aq":"ATA",	"ag":"ATG",	"ar":"ARG",	"am":"ARM",	"aw":"ABW",	"au":"AUS",	"at":"AUT",	"az":"AZE",	"bs":"BHS",	"bh":"BHR",	"bd":"BGD",	"bb":"BRB",	"by":"BLR",	"be":"BEL",	"bz":"BLZ",	"bj":"BEN",	"bm":"BMU",	"bt":"BTN",	"bo":"BOL",	"bq":"BES",	"ba":"BIH",	"bw":"BWA",	"bv":"BVT",	"br":"BRA",	"io":"IOT",	"bn":"BRN",	"bg":"BGR",	"bf":"BFA",	"bi":"BDI",	"kh":"KHM",	"cm":"CMR",	"ca":"CAN",	"cv":"CPV",	"ky":"CYM",	"cf":"CAF",	"td":"TCD",	"cl":"CHL",	"cn":"CHN",	"cx":"CXR",	"cc":"CCK",	"co":"COL",	"km":"COM",	"cg":"COG",	"cd":"COD",	"ck":"COK",	"cr":"CRI",	"hr":"HRV",	"cu":"CUB",	"cw":"CUW",	"cy":"CYP",	"cz":"CZE",	"ci":"CIV",	"dk":"DNK",	"dj":"DJI",	"dm":"DMA",	"do":"DOM",	"ec":"ECU",	"eg":"EGY",	"sv":"SLV",	"gq":"GNQ",	"er":"ERI",	"ee":"EST",	"et":"ETH",	"fk":"FLK",	"fo":"FRO",	"fj":"FJI",	"fi":"FIN",	"fr":"FRA",	"gf":"GUF",	"pf":"PYF",	"tf":"ATF",	"ga":"GAB",	"gm":"GMB",	"ge":"GEO",	"de":"DEU",	"gh":"GHA",	"gi":"GIB",	"gr":"GRC",	"gl":"GRL",	"gd":"GRD",	"gp":"GLP",	"gu":"GUM",	"gt":"GTM",	"gg":"GGY",	"gn":"GIN",	"gw":"GNB",	"gy":"GUY",	"ht":"HTI",	"hm":"HMD",	"va":"VAT",	"hn":"HND",	"hk":"HKG",	"hu":"HUN",	"is":"ISL",	"in":"IND",	"id":"IDN",	"ir":"IRN",	"iq":"IRQ",	"ie":"IRL",	"im":"IMN",	"il":"ISR",	"it":"ITA",	"jm":"JAM",	"jp":"JPN",	"je":"JEY",	"jo":"JOR",	"kz":"KAZ",	"ke":"KEN",	"ki":"KIR",	"kp":"PRK",	"kr":"KOR",	"kw":"KWT",	"kg":"KGZ",	"la":"LAO",	"lv":"LVA",	"lb":"LBN",	"ls":"LSO",	"lr":"LBR",	"ly":"LBY",	"li":"LIE",	"lt":"LTU",	"lu":"LUX",	"mo":"MAC",	"mk":"MKD",	"mg":"MDG",	"mw":"MWI",	"my":"MYS",	"mv":"MDV",	"ml":"MLI",	"mt":"MLT",	"mh":"MHL",	"mq":"MTQ",	"mr":"MRT",	"mu":"MUS",	"yt":"MYT",	"mx":"MEX",	"fm":"FSM",	"md":"MDA",	"mc":"MCO",	"mn":"MNG",	"me":"MNE",	"ms":"MSR",	"ma":"MAR",	"mz":"MOZ",	"mm":"MMR",	"na":"NAM",	"nr":"NRU",	"np":"NPL",	"nl":"NLD",	"nc":"NCL",	"nz":"NZL",	"ni":"NIC",	"ne":"NER",	"ng":"NGA",	"nu":"NIU",	"nf":"NFK",	"mp":"MNP",	"no":"NOR",	"om":"OMN",	"pk":"PAK",	"pw":"PLW",	"ps":"PSE",	"pa":"PAN",	"pg":"PNG",	"py":"PRY",	"pe":"PER",	"ph":"PHL",	"pn":"PCN",	"pl":"POL",	"pt":"PRT",	"pr":"PRI",	"qa":"QAT",	"ro":"ROU",	"ru":"RUS",	"rw":"RWA",	"re":"REU",	"bl":"BLM",	"sh":"SHN",	"kn":"KNA",	"lc":"LCA",	"mf":"MAF",	"pm":"SPM",	"vc":"VCT",	"ws":"WSM",	"sm":"SMR",	"st":"STP",	"sa":"SAU",	"sn":"SEN",	"rs":"SRB",	"sc":"SYC",	"sl":"SLE",	"sg":"SGP",	"sx":"SXM",	"sk":"SVK",	"si":"SVN",	"sb":"SLB",	"so":"SOM",	"za":"ZAF",	"gs":"SGS",	"ss":"SSD",	"es":"ESP",	"lk":"LKA",	"sd":"SDN",	"sr":"SUR",	"sj":"SJM",	"sz":"SWZ",	"se":"SWE",	"ch":"CHE",	"sy":"SYR",	"tw":"TWN",	"tj":"TJK",	"tz":"TZA",	"th":"THA",	"tl":"TLS",	"tg":"TGO",	"tk":"TKL",	"to":"TON",	"tt":"TTO",	"tn":"TUN",	"tr":"TUR",	"tm":"TKM",	"tc":"TCA",	"tv":"TUV",	"ug":"UGA",	"ua":"UKR",	"ae":"ARE",	"gb":"GBR",	"us":"USA",	"um":"UMI",	"uy":"URY",	"uz":"UZB",	"vu":"VUT",	"ve":"VEN",	"vn":"VNM",	"vg":"VGB",	"vi":"VIR",	"wf":"WLF",	"eh":"ESH",	"ye":"YEM",	"zm":"ZMB",	"zw":"ZWE"}
    geolocator = Nominatim(timeout=3)
    #location to be chosen so map will zoom on it 
    US_COORD = [39,-95]

    # Build map 
    map_main = folium.Map(location=US_COORD,
                          zoom_start=2, 
                          tiles='cartodbpositron', 
                          width=900, 
                          height=500)         


    location_list=[]
    location_score_list=[]
    location_tweet_count=[]
    country_sentiment=[]

    for i, tweet in enumerate(reader):
        w_progress_bar.value += 1/len(reader)
        location_json = tweet['tweet']['user']['location']
        #print("location_json : {0}".format(location_json)) #affichage pour debug
        if (location_json!= ''):
            geolocation = geolocator.geocode(location_json)
            if geolocation != None:

                popup_image ='''<img src="''' + tweet['tweet']['user']['profile_image_url'] + '''" alt="pic " />'''
                popup_text = '<b>'+tweet['tweet']['user']['name'].replace("'", r'\"')+'</b>' + ' tweeted : '+ '<i>' + tweet['tweet']['full_text'].replace("'", r'\"') + '</i>'
                popup = '<center>' + popup_image + '<br\><br\>' + popup_text + '</center>'
                location_lat_long = [geolocation.latitude + random.gauss(0,0000.1), geolocation.longitude + random.gauss(0,000.1)]
                reverse=geolocator.reverse(location_lat_long)
                country=d[reverse.raw['address']['country_code']]

                note=0
                if tweet['result'] == 'Positif':
                    folium.Marker(location_lat_long,                               
                                  popup= popup,
                                  #popup= folium.Popup(test, max_width=2650),
                                  icon=folium.Icon(color='green')
                                 ).add_to(map_main)
                    note+=1
                elif tweet['result'] == 'Negatif':
                    folium.Marker(location_lat_long,   
                                  popup= popup_text,
                                  icon=folium.Icon(color='red')
                                 ).add_to(map_main)  
                    note-=1
                elif tweet['result'] == 'Neutre':
                    folium.Marker(location_lat_long,   
                                  popup= popup,
                                  icon=folium.Icon(color='lightgray')
                                 ).add_to(map_main)
                if (country != None)&(country in location_list):
                    location_score_list[location_list.index(country)]+=note
                    location_tweet_count[location_list.index(country)]+=1
                if (country not in location_list)&(country != None):
                    location_list.append(country)
                    location_score_list.append(0)
                    location_tweet_count.append(0)

    for i in range(len(location_tweet_count)):
        if (location_tweet_count[i]!=0):
            location_score_list[i] =location_score_list[i]/location_tweet_count[i]
    #Define heat_map (aka map_color)
    country_geo = os.path.join('countries.geo.json')
    country_sentiment=pd.DataFrame(
        {'Country': location_list,
         'Sentiment': location_score_list,
        })

    map_color = folium.Map(location=US_COORD, 
                          zoom_start=2, 
                          tiles='cartodbpositron', 
                          width=900, 
                          height=500)     
    map_color.choropleth(
     geo_data=country_geo,
     name='Sentiment',
     data=country_sentiment,
     columns=['Country', 'Sentiment'],
     key_on='feature.id',
     fill_color='PuBuGn',
     fill_opacity=0.9,
     line_opacity=0.2,
     legend_name='Sentiment'
    )
    folium.LayerControl().add_to(map_color)

Visualisation des tweets sur carte

In [6]:
def plot_map(b):
    clear_output()
    display(w_button_map)
    display(map_main)

Echelle de pays par sentiments

In [7]:
def plot_heatmap(b):
    clear_output()
    display(w_button_heatmap)
    display(map_color)

Treemap chart

In [8]:
def plot_treemap(b):
    clear_output()
    display(w_button_treemap)
    y=[0,0,0];
    with open("tweets-database/" + OUTPUT_TWEET_FILENAME, 'rb') as f:
        for tweet in json_lines.reader(f):
            if tweet['result'] == 'Positif':
                y[0]+=1
            elif tweet['result'] == 'Negatif':
                y[1]+=1
            elif tweet['result'] == 'Neutre':
                y[2]+=1

    x = ["Positif", "Négatif", "Neutre"];
    x = [x[i] + " (" + str(y[i]) + ")" for i in range(3)]

    colors=["#87AF41","#F63E02","#FAA613"];
    if 0 in y:
        k=y.index(0)
        del x[k]
        del y[k]
        del colors[k]
    if 0 in y:
        k=y.index(0)
        del x[k]
        del y[k]
        del colors[k]

    squarify.plot(sizes=y, label=x, alpha=.7 ,color=colors)
    plt.axis('off')
    plt.show()

Liste des tweets

In [9]:
def plot_tweet_list(b):
    clear_output()
    display(w_button_tweet_list)
    data=[["Score","Tweet"]]
    with open("tweets-database/" + OUTPUT_TWEET_FILENAME, 'rb') as f:
        for tweet in json_lines.reader(f):
            data.append([tweet["result"],tweet["tweet"]["full_text"]])
    t = Texttable()
    t.add_rows(data)
    print (t.draw())

In [10]:
w_button_list = widgets.Button(description="Display list of tweets")
w_button_list.on_click(plot_tweet_list)

w_button_map = widgets.Button(description="Plot map")
w_button_map.on_click(plot_map)

w_button_heatmap = widgets.Button(description="Plot heat map")
w_button_heatmap.on_click(plot_heatmap)

w_button_treemap = widgets.Button(description="Plot treemap")
w_button_treemap.on_click(plot_treemap)

In [11]:
display(w_count)
display(w_dates)
display(w_keyword)
display(w_rts)
display(w_language)
display(w_type)
display(w_coordinates)
display(w_radius)
display(button_query)
display(w_progress_bar)
display(w_progress_legend)

IntSlider(value=100, description='Nb of tweets', max=1000)

SelectionRangeSlider(description='Date range', index=(0, 9), options=('18/03', '19/03', '20/03', '21/03', '22/03', '23/03', '24/03', '25/03', '26/03', '27/03'), value=('18/03', '27/03'))

Text(value='#Trump', description='Keyword', placeholder='keyword or hashtag')

Checkbox(value=False, description='Exclude retweets ?')

ToggleButtons(button_style='danger', description='Language', options=('en', 'fr', 'de'), tooltips=['English', 'French', 'German'], value='en')

...

Text(value='', description='Coordinates', placeholder='Coordinates')

IntText(value=100, description='Radius (km)')

Button(description='Perform the query and the analysis', style=ButtonStyle())

IntProgress(value=0, description='Loading:', max=7)

HTML(value='Waiting for input ... ', description='Action : ', placeholder='Some HTML')

In [12]:
display(w_button_list)

Button(description='Display list of tweets', style=ButtonStyle())

In [13]:
display(w_button_map)

Button(description='Plot map', style=ButtonStyle())

In [14]:
display(w_button_treemap)

Button(description='Plot treemap', style=ButtonStyle())

In [15]:
display(w_button_heatmap)

Button(description='Plot heat map', style=ButtonStyle())